In [ ]:
##Import images and calibrate


In [1]:
import cv2
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

output_img_path = "./output_images/"

source_path = "./camera_cal/"
objp = np.zeros((6*9, 3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

objpoints = []
imgpoints = []

imgs = glob.glob('./camera_cal/calibration*.jpg')
for (idx, filename) in enumerate(imgs):
    img = cv2.imread(filename)
    img_size = img.shape[1::-1]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

ret, calibration_mtx, calibration_dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
print(calibration_mtx)

[[  1.15396093e+03   0.00000000e+00   6.69705357e+02]
 [  0.00000000e+00   1.14802496e+03   3.85656234e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]


In [ ]:
##check to see if we actually found corners, save an image
img = cv2.imread('./camera_cal/calibration9.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
if ret == True:
    corner_img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
    plt.imshow(img)
    plt.show()
    cv2.imwrite(output_img_path + "corners9.jpg", img)

img_size = img.shape[1::-1]

In [ ]:
img = cv2.imread('./camera_cal/calibration8.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

undist = cv2.undistort(img, calibration_mtx, calibration_dist, None, calibration_mtx)
result = undist

plt.imshow(result)
plt.show()
cv2.imwrite(output_img_path + "undist8.jpg", result)

img_size = img.shape[1::-1]

In [4]:
##helper functions, given from lessons
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0,255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, sobel_kernel))
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return binary_output

def color_threshold(img, sthresh=(0,255), vthresh=(0,255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= sthresh[0]) & (s_channel <= sthresh[1])] = 1
    
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >= vthresh[0]) & (v_channel <= vthresh[1])] = 1
    
    out = np.zeros_like(s_channel)
    out[(s_binary == 1) & (v_binary == 1)] = 1
    return out
    
def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

    

In [5]:
class tracker():
    
    def __init__(self, window_width, window_height, margin, ym = 1, xm = 1, smooth_factor = 15):
        self.recent_centers = []
        
        self.window_width = window_width
        self.window_height = window_height
        self.margin = margin
        self.ym_per_pix = ym
        self.xm_per_pix = xm
        self.smooth = smooth_factor
    
    def find_window_centroids(self, image):

        window_centroids = []
        window_width = self.window_width
        window_height = self.window_height
        margin = self.margin

        window = np.ones(window_width) # Create our window template that we will use for convolutions

        # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
        # and then np.convolve the vertical image slice with the window template 

        # Sum quarter bottom of image to get slice, could use a different ratio
        l_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)], axis=0)
        l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
        r_sum = np.sum(image[int(3*image.shape[0]/4):,int(image.shape[1]/2):], axis=0)
        r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(image.shape[1]/2)

        # Add what we found for the first layer
        window_centroids.append((l_center,r_center))

        # Go through each layer looking for max pixel locations
        for level in range(1,(int)(image.shape[0]/window_height)):
            # convolve the window into the vertical slice of the image
            image_layer = np.sum(image[int(image.shape[0]-(level+1)*window_height):int(image.shape[0]-level*window_height),:], axis=0)
            conv_signal = np.convolve(window, image_layer)
            # Find the best left centroid by using past left center as a reference
            # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
            offset = window_width/2
            l_min_index = int(max(l_center+offset-margin,0))
            l_max_index = int(min(l_center+offset+margin,image.shape[1]))
            l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
            # Find the best right centroid by using past right center as a reference
            r_min_index = int(max(r_center+offset-margin,0))
            r_max_index = int(min(r_center+offset+margin,image.shape[1]))
            r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
            # Add what we found for that layer
            window_centroids.append((l_center,r_center))

        self.recent_centers.append(window_centroids)

        return np.average(self.recent_centers[-self.smooth:], axis = 0)
    

In [ ]:
## Undistort camera

temp_imgs = './temp_imgs/'
dummy_im = cv2.imread('./test_images/test3.jpg')
std_im_size = dummy_im.shape[1::-1]

bot_width = .76
mid_width = .08
height_pct = .62
bottom_trim = .935
right_shift = 0.015

##show the searching polygon
top_left = [std_im_size[0]*(.5-mid_width/2),std_im_size[1]*height_pct]
top_right = [std_im_size[0]*(.5+mid_width/2), std_im_size[1]*height_pct]
bottom_right = [std_im_size[0]*(.5+bot_width/2), std_im_size[1]*bottom_trim]
bottom_left = [std_im_size[0]*(.5-bot_width/2), std_im_size[1]*bottom_trim]

top_left_shift = [std_im_size[0]*(.5-mid_width/2) + right_shift*std_im_size[0],std_im_size[1]*height_pct]
top_right_shift = [std_im_size[0]*(.5+mid_width/2) + right_shift*std_im_size[0], std_im_size[1]*height_pct]
bottom_right_shift = [std_im_size[0]*(.5+bot_width/2) + right_shift*std_im_size[0], std_im_size[1]*bottom_trim]
bottom_left_shift = [std_im_size[0]*(.5-bot_width/2) + right_shift*std_im_size[0], std_im_size[1]*bottom_trim]

corners = [top_left, top_right, bottom_right, bottom_left]
corners_shift = [top_left_shift,top_right_shift, bottom_right_shift, bottom_left_shift]

test_img_path = './test_images/'
imgs = glob.glob(test_img_path + 'test*.jpg')

for idx, filename in enumerate(imgs):
        
    img = cv2.imread(filename)
    
    undist = cv2.undistort(img, calibration_mtx, calibration_dist, None, calibration_mtx)
    
    preprocess = np.zeros_like(img[:,:,0])
    gradx = abs_sobel_thresh(img, orient='x', thresh=(12,255))
    grady = abs_sobel_thresh(img, orient='y', thresh=(25,255))
    c_binary = color_threshold(img, sthresh=(100, 255), vthresh=(50,255))
    preprocess[((gradx == 1) & (grady == 1) | (c_binary == 1))] = 255
    result = preprocess

    
#     ##Img.shape = (y,x,z)
    img_size = img.shape[1::-1]

    src = np.float32(corners_shift)
    offset = img_size[0] *.25 
    dst = np.float32([[offset, 0], [img_size[0]-offset, 0], [img_size[0] - offset, img_size[1]], [offset , img_size[1]]])
                      
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(preprocess, M, img_size, flags=cv2.INTER_LINEAR)
    
    window_width = 25
    window_height = 80
    margin = 25
    xm_per_pix = 4/384
    ym_per_pix = 10/720
    
    curve_centers = tracker(window_width = window_width, window_height = window_height, margin =  25, ym = ym_per_pix, xm = xm_per_pix, smooth_factor = 15)
    
    window_centroids = curve_centers.find_window_centroids(warped)
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)
    
    leftx = []
    rightx = []

    # Go through each level and draw the windows
    for level in range(0,len(window_centroids)):
        # Window_mask is a function to draw window areas        
        l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
        r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
        
        leftx.append(window_centroids[level][0])
        rightx.append(window_centroids[level][1])

        # Add graphic points from window mask here to total pixels found 
        l_points[(l_points == 255) | (l_mask == 1)] = 255
        r_points[(r_points == 255) | (r_mask == 1)] = 255

    # Draw the results
    template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage= np.array(cv2.merge((warped, warped, warped)), np.uint8)# making the original road pixels 3 color channels
    result = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
    
    yvals = range(0, warped.shape[0])
    res_yvals = np.arange(warped.shape[0] - (window_height/2), 0, -window_height)
    
    left_fit = np.polyfit(res_yvals, leftx, 2)
    left_fitx = left_fit[0]*yvals*yvals + left_fit[1]*yvals + left_fit[2]
    left_fitx = np.array(left_fitx, np.int32)
    
    right_fit = np.polyfit(res_yvals, rightx, 2)
    right_fitx = right_fit[0]*yvals*yvals + right_fit[1]*yvals + right_fit[2]
    left_fitx = np.array(left_fitx, np.int32)
    
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    pts_left = np.array([np.transpose(np.vstack([left_fitx,yvals]))])
    pts_right = np.array([np.transpose(np.vstack([right_fitx,yvals]))])
    pts = np.hstack((pts_left, pts_right)) 
    np.set_printoptions(threshold=np.inf)
    
    pts = pts[0,0::]
    
    pts[720::] = pts[720::][::-1]
        
    ##draw the polygon, unwarp it and add it back    
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0]))
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    #radius of curvature
    curve_fit_cr_l = np.polyfit(np.array(res_yvals, np.float32)*ym_per_pix, np.array(leftx, np.float32)*xm_per_pix, 2)
    curve_fit_cr_r = np.polyfit(np.array(res_yvals, np.float32)*ym_per_pix, np.array(rightx, np.float32)*xm_per_pix, 2)
    
    curverad_l = ((1 + (2*curve_fit_cr_l[0]*yvals[-1]*ym_per_pix + curve_fit_cr_l[1])**2)**1.5) /np.absolute(2*curve_fit_cr_l[0])
    curverad_r = ((1 + (2*curve_fit_cr_r[0]*yvals[-1]*ym_per_pix + curve_fit_cr_r[1])**2)**1.5) /np.absolute(2*curve_fit_cr_r[0])
    
    curverad = (curverad_l + curverad_r)/2
    
    #different from center
    camera_center = (left_fitx[-1] + right_fitx[-1])/2
    center_dif = (camera_center - warped.shape[1]/2)*xm_per_pix
    side_pos = 'left'
    if center_dif <=0:
        side_pos = 'right'
        
    cv2.putText(result, 'Radius of Curvature = '+str(round(curverad, 3)) + '(m)', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.putText(result, 'Vehicle is ' +str(abs(round(center_dif, 3))) + 'm ' + side_pos + ' of center', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    
        
    write_name = temp_imgs +'frame_test' + str(idx)+'.jpg'
    cv2.imwrite(write_name, result)
    

In [ ]:
### undistorted images

test_img_path = './test_images/'
imgs = glob.glob(test_img_path + 'test*.jpg')
for idx, filename in enumerate(imgs):
    img = cv2.imread(filename)
    
    undist = cv2.undistort(img, calibration_mtx, calibration_dist, None, calibration_mtx)
    result = undist

    write_name = temp_imgs +'undist' + str(idx)+'.jpg'
    cv2.imwrite(write_name, result)
    

In [ ]:
## binary images
test_img_path = './test_images/'
imgs = glob.glob(test_img_path + 'test*.jpg')
for idx, filename in enumerate(imgs):
    img = cv2.imread(filename)
    
    undist = cv2.undistort(img, calibration_mtx, calibration_dist, None, calibration_mtx)
    preprocess = np.zeros_like(img[:,:,0])
    gradx = abs_sobel_thresh(img, orient='x', thresh=(12,255))
    grady = abs_sobel_thresh(img, orient='y', thresh=(25,255))
    c_binary = color_threshold(img, sthresh=(100, 255), vthresh=(50,255))
    preprocess[((gradx == 1) & (grady == 1) | (c_binary == 1))] = 255
    result = preprocess

    write_name = temp_imgs +'binary' + str(idx)+'.jpg'
    cv2.imwrite(write_name, result)


In [ ]:
###undistorted with AOI polygon

test_img_path = './test_images/'
imgs = glob.glob(test_img_path + 'test*.jpg')
for idx, filename in enumerate(imgs):
    img = cv2.imread(filename)
    
    undist = cv2.undistort(img, calibration_mtx, calibration_dist, None, calibration_mtx)

    lines = cv2.polylines(undist, np.int32([corners_shift]), True, (0,0,255), 5)

    result = lines

    write_name = temp_imgs +'polygon' + str(idx)+'.jpg'
    cv2.imwrite(write_name, result)


In [ ]:
## Warped images
test_img_path = './test_images/'
imgs = glob.glob(test_img_path + 'test*.jpg')
for idx, filename in enumerate(imgs):
    img = cv2.imread(filename)
    
    undist = cv2.undistort(img, calibration_mtx, calibration_dist, None, calibration_mtx)
    preprocess = np.zeros_like(img[:,:,0])
    gradx = abs_sobel_thresh(img, orient='x', thresh=(12,255))
    grady = abs_sobel_thresh(img, orient='y', thresh=(25,255))
    c_binary = color_threshold(img, sthresh=(100, 255), vthresh=(50,255))
    preprocess[((gradx == 1) & (grady == 1) | (c_binary == 1))] = 255
    
    img_size = img.shape[1::-1]

    src = np.float32(corners_shift)
    offset = img_size[0] *.25 
    dst = np.float32([[offset, 0], [img_size[0]-offset, 0], [img_size[0] - offset, img_size[1]], [offset , img_size[1]]])
                      
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(preprocess, M, img_size, flags=cv2.INTER_LINEAR)
    result = warped

    write_name = temp_imgs +'warped' + str(idx)+'.jpg'
    cv2.imwrite(write_name, result)

In [6]:
####Video
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def process_image(img):
    std_im_size = img.shape[1::-1]
    
    bot_width = .76
    mid_width = .08
    height_pct = .62
    bottom_trim = .935
    right_shift = 0.015

    ##show the searching polygon
    top_left = [std_im_size[0]*(.5-mid_width/2),std_im_size[1]*height_pct]
    top_right = [std_im_size[0]*(.5+mid_width/2), std_im_size[1]*height_pct]
    bottom_right = [std_im_size[0]*(.5+bot_width/2), std_im_size[1]*bottom_trim]
    bottom_left = [std_im_size[0]*(.5-bot_width/2), std_im_size[1]*bottom_trim]

    top_left_shift = [std_im_size[0]*(.5-mid_width/2) + right_shift*std_im_size[0],std_im_size[1]*height_pct]
    top_right_shift = [std_im_size[0]*(.5+mid_width/2) + right_shift*std_im_size[0], std_im_size[1]*height_pct]
    bottom_right_shift = [std_im_size[0]*(.5+bot_width/2) + right_shift*std_im_size[0], std_im_size[1]*bottom_trim]
    bottom_left_shift = [std_im_size[0]*(.5-bot_width/2) + right_shift*std_im_size[0], std_im_size[1]*bottom_trim]

    corners = [top_left, top_right, bottom_right, bottom_left]
    corners_shift = [top_left_shift,top_right_shift, bottom_right_shift, bottom_left_shift]

    undist = cv2.undistort(img, calibration_mtx, calibration_dist, None, calibration_mtx)

    preprocess = np.zeros_like(img[:,:,0])
    gradx = abs_sobel_thresh(img, orient='x', thresh=(12,255))
    grady = abs_sobel_thresh(img, orient='y', thresh=(25,255))
    c_binary = color_threshold(img, sthresh=(100, 255), vthresh=(50,255))
    preprocess[((gradx == 1) & (grady == 1) | (c_binary == 1))] = 255
    result = preprocess
#     ##Img.shape = (y,x,z)
    img_size = std_im_size

    src = np.float32(corners)
    offset = img_size[0] *.25 
    dst = np.float32([[offset, 0], [img_size[0]-offset, 0], [img_size[0] - offset, img_size[1]], [offset , img_size[1]]])

    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(preprocess, M, img_size, flags=cv2.INTER_LINEAR)

   
    window_centroids = curve_centers.find_window_centroids(warped)
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)

    leftx = []
    rightx = []

    # Go through each level and draw the windows
    for level in range(0,len(window_centroids)):
        # Window_mask is a function to draw window areas        
        l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
        r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)

        leftx.append(window_centroids[level][0])
        rightx.append(window_centroids[level][1])

        # Add graphic points from window mask here to total pixels found 
        l_points[(l_points == 255) | (l_mask == 1)] = 255
        r_points[(r_points == 255) | (r_mask == 1)] = 255

    # Draw the results
    template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage= np.array(cv2.merge((warped, warped, warped)), np.uint8)# making the original road pixels 3 color channels
    result = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results

    yvals = range(0, warped.shape[0])
    res_yvals = np.arange(warped.shape[0] - (window_height/2), 0, -window_height)

    left_fit = np.polyfit(res_yvals, leftx, 2)
    left_fitx = left_fit[0]*yvals*yvals + left_fit[1]*yvals + left_fit[2]
    left_fitx = np.array(left_fitx, np.int32)

    right_fit = np.polyfit(res_yvals, rightx, 2)
    right_fitx = right_fit[0]*yvals*yvals + right_fit[1]*yvals + right_fit[2]
    left_fitx = np.array(left_fitx, np.int32)

    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    pts_left = np.array([np.transpose(np.vstack([left_fitx,yvals]))])
    pts_right = np.array([np.transpose(np.vstack([right_fitx,yvals]))])
    pts = np.hstack((pts_left, pts_right)) 
    np.set_printoptions(threshold=np.inf)

    pts = pts[0,0::]

    pts[720::] = pts[720::][::-1]

    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0]))
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)

    curve_fit_cr_l = np.polyfit(np.array(res_yvals, np.float32)*ym_per_pix, np.array(leftx, np.float32)*xm_per_pix, 2)
    curve_fit_cr_r = np.polyfit(np.array(res_yvals, np.float32)*ym_per_pix, np.array(rightx, np.float32)*xm_per_pix, 2)
    
    curverad_l = ((1 + (2*curve_fit_cr_l[0]*yvals[-1]*ym_per_pix + curve_fit_cr_l[1])**2)**1.5) /np.absolute(2*curve_fit_cr_l[0])
    curverad_r = ((1 + (2*curve_fit_cr_r[0]*yvals[-1]*ym_per_pix + curve_fit_cr_r[1])**2)**1.5) /np.absolute(2*curve_fit_cr_r[0])
    
    curverad = (curverad_l + curverad_r)/2
    
    camera_center = (left_fitx[-1] + right_fitx[-1])/2
    center_dif = (camera_center - warped.shape[1]/2)*xm_per_pix
    side_pos = 'left'
    if center_dif <=0:
        side_pos = 'right'

    cv2.putText(result, 'Radius of Curvature = '+str(round(curverad, 3)) + '(m)', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.putText(result, 'Vehicle is ' +str(abs(round(center_dif, 3))) + 'm ' + side_pos + ' of center', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    return result
        
    

In [7]:
output_video = 'test_video.mp4'
input_video = 'project_video.mp4'

window_width = 25
window_height = 80
margin = 25
xm_per_pix = 3.7/700
ym_per_pix = 30/720

curve_centers = tracker(window_width = window_width, window_height = window_height, margin =  25, ym = ym_per_pix, xm = xm_per_pix, smooth_factor = 10)

clip1 = VideoFileClip(input_video)
video_clip = clip1.fl_image(process_image)
video_clip.write_videofile(output_video, audio=False)


[MoviePy] >>>> Building video test_video.mp4
[MoviePy] Writing video test_video.mp4


100%|█████████▉| 1260/1261 [02:54<00:00,  8.27it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_video.mp4 



In [ ]:
output_video = 'challenge_test_video.mp4'
input_video = 'challenge_video.mp4'

window_width = 25
window_height = 80
margin = 25
xm_per_pix = 4/384
ym_per_pix = 10/720

curve_centers = tracker(window_width = window_width, window_height = window_height, margin =  25, ym = ym_per_pix, xm = xm_per_pix, smooth_factor = 10)


clip1 = VideoFileClip(input_video)
video_clip = clip1.fl_image(process_challenge_image)
video_clip.write_videofile(output_video, audio=False)


In [ ]:
####Video
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def process__challenge_image(img):
    std_im_size = img.shape[1::-1]
    
    bot_width = .76
    mid_width = .08
    height_pct = .62
    bottom_trim = .935
    right_shift = 0.015

    ##show the searching polygon
    top_left = [std_im_size[0]*(.5-mid_width/2),std_im_size[1]*height_pct]
    top_right = [std_im_size[0]*(.5+mid_width/2), std_im_size[1]*height_pct]
    bottom_right = [std_im_size[0]*(.5+bot_width/2), std_im_size[1]*bottom_trim]
    bottom_left = [std_im_size[0]*(.5-bot_width/2), std_im_size[1]*bottom_trim]

    top_left_shift = [std_im_size[0]*(.5-mid_width/2) + right_shift*std_im_size[0],std_im_size[1]*height_pct]
    top_right_shift = [std_im_size[0]*(.5+mid_width/2) + right_shift*std_im_size[0], std_im_size[1]*height_pct]
    bottom_right_shift = [std_im_size[0]*(.5+bot_width/2) + right_shift*std_im_size[0], std_im_size[1]*bottom_trim]
    bottom_left_shift = [std_im_size[0]*(.5-bot_width/2) + right_shift*std_im_size[0], std_im_size[1]*bottom_trim]

    corners = [top_left, top_right, bottom_right, bottom_left]
    corners_shift = [top_left_shift,top_right_shift, bottom_right_shift, bottom_left_shift]

    undist = cv2.undistort(img, calibration_mtx, calibration_dist, None, calibration_mtx)

    preprocess = np.zeros_like(img[:,:,0])
    gradx = abs_sobel_thresh(img, orient='x', thresh=(12,255))
    grady = abs_sobel_thresh(img, orient='y', thresh=(25,255))
    c_binary = color_threshold(img, sthresh=(100, 255), vthresh=(50,255))
    preprocess[((gradx == 1) & (grady == 1) | (c_binary == 1))] = 255
    result = preprocess
#     ##Img.shape = (y,x,z)
    img_size = std_im_size

    src = np.float32(corners)
    offset = img_size[0] *.25 
    dst = np.float32([[offset, 0], [img_size[0]-offset, 0], [img_size[0] - offset, img_size[1]], [offset , img_size[1]]])

    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(preprocess, M, img_size, flags=cv2.INTER_LINEAR)

   
    window_centroids = curve_centers.find_window_centroids(warped)
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)

    leftx = []
    rightx = []

    # Go through each level and draw the windows
    for level in range(0,len(window_centroids)):
        # Window_mask is a function to draw window areas        
        l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
        r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)

        leftx.append(window_centroids[level][0])
        rightx.append(window_centroids[level][1])

        # Add graphic points from window mask here to total pixels found 
        l_points[(l_points == 255) | (l_mask == 1)] = 255
        r_points[(r_points == 255) | (r_mask == 1)] = 255

    # Draw the results
    template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage= np.array(cv2.merge((warped, warped, warped)), np.uint8)# making the original road pixels 3 color channels
    result = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results

    yvals = range(0, warped.shape[0])
    res_yvals = np.arange(warped.shape[0] - (window_height/2), 0, -window_height)

    left_fit = np.polyfit(res_yvals, leftx, 2)
    left_fitx = left_fit[0]*yvals*yvals + left_fit[1]*yvals + left_fit[2]
    left_fitx = np.array(left_fitx, np.int32)

    right_fit = np.polyfit(res_yvals, rightx, 2)
    right_fitx = right_fit[0]*yvals*yvals + right_fit[1]*yvals + right_fit[2]
    left_fitx = np.array(left_fitx, np.int32)

    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    pts_left = np.array([np.transpose(np.vstack([left_fitx,yvals]))])
    pts_right = np.array([np.transpose(np.vstack([right_fitx,yvals]))])
    pts = np.hstack((pts_left, pts_right)) 
    np.set_printoptions(threshold=np.inf)

    pts = pts[0,0::]

    pts[720::] = pts[720::][::-1]

    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0]))
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)

    curve_fit_cr = np.polyfit(np.array(res_yvals, np.float32)*ym_per_pix, np.array(leftx, np.float32)*xm_per_pix, 2)
    curverad = ((1 + (2*curve_fit_cr[0]*yvals[-1]*ym_per_pix + curve_fit_cr[1])**2)**1.5) /np.absolute(2*curve_fit_cr[0])

    camera_center = (left_fitx[-1] + right_fitx[-1])/2
    center_dif = (camera_center - warped.shape[1]/2)*xm_per_pix
    side_pos = 'left'
    if center_dif <=0:
        side_pos = 'right'

    cv2.putText(result, 'Radius of Curvature = '+str(round(curverad, 3)) + '(m)', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.putText(result, 'Vehicle is ' +str(abs(round(center_dif, 3))) + 'm ' + side_pos + ' of center', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    return result
        
    